# はじめに
[Word Embeddings](https://colab.research.google.com/drive/1tttDqgnWL9yJtmlOFXJqA-BjQ1Pyfpax#scrollTo=ws7hh97DATIO)のアレンジです。
就業規則を答えるBOTを作成します。
就業規則には練習として厚生労働省出版の[モデル就業規則](https://www.mhlw.go.jp/stf/seisakunitsuite/bunya/koyou_roudou/roudoukijun/zigyonushi/model/index.html)を使用します。

Jupyter NotebookはGoogle Colab上で公開しております。
https://colab.research.google.com/drive/1TKAmnBVJVnNmZDYm6-fQOKFBWPtp0TN4?usp=sharing

# モジュールの準備

In [2]:
!pip install tiktoken openai -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 10.8 MB/s eta 0:00:00


In [3]:
import openai
from openai.embeddings_utils import get_embedding, cosine_similarity
import tiktoken
import pandas as pd
import numpy as np
from getpass import getpass

OpenAI APIトークンの設定

In [4]:
openai.api_key = getpass()

··········


# データの準備
下の方で学習済みのデータを公開していますので、データの準備手順に興味のない方は `データを永続化させるためにGoogleDriveへ接続`の項まで飛ばしてください。

モデル就業規則のpdfをcurlで取得してpdftotextでテキスト化します。
https://www.mhlw.go.jp/content/001018385.pdf

```sh
$ curl -fsSLo temp.pdf https://www.mhlw.go.jp/content/001018385.pdf  | pdftotext -nopgbrk -raw temp.pdf
```

上記コマンドを実行するとカレントディレクトリ上に`temp.pdf`と`temp.txt`が作成されており、それぞれモデル就業規則のPDFとテキストデータが入っています。


In [ ]:
model_code = """
１ モデル就業規則
令和４年 11 月版
厚生労働省労働基準局監督課
２ はじめに
１ 就業規則の意義
労働者が安心して働ける明るい職場を作ることは、事業規模や業種を問わず、すべての事業場にとって重要なことです。そのためには、あらかじめ就業規則で労働時間や賃金をはじめ、人事・服務規律など、労働者の労働条件や待遇の基準をはっきりと定め、労使間でトラブルが生じないようにしておくことが大切です。
２ 就業規則の内容
就業規則に記載する事項には、必ず記載しなければならない事項（以下「絶対的必要記載事項」といいます。）と、各事業場内でルールを定める場合には記載しなければならない事項（以下「相対的必要記載事項」といいます。）があります（労働基準法（昭和２２年法律第４９号。以下「労基法」といいます。）第８９条）。このほか、使用者において任意に記載し得る事項もあります。
絶対的必要記載事項は次のとおりです。
（１） 労働時間関係 始業及び終業の時刻、休憩時間、休日、休暇並びに労働者を２組以上に分けて交替に就業させる場合においては就業時転換に関する事項
（２） 賃金関係 賃金の決定、計算及び支払の方法、賃金の締切り及び支払の時期並びに昇給に関する事項
（３） 退職関係 退職に関する事項（解雇の事由を含みます。）相対的必要記載事項は次のとおりです。
（１） 退職手当関係 適用される労働者の範囲、退職手当の決定、計算及び支払の方法並びに退職手当の支払の時期に関する事項
（２） 臨時の賃金・最低賃金額関係 臨時の賃金等(退職手当を除きます。)及び最低賃金額に関する事項
（３） 費用負担関係
３ 労働者に食費、作業用品その他の負担をさせることに関する事項
（４） 安全衛生関係 安全及び衛生に関する事項
（５） 職業訓練関係 職業訓練に関する事項
（６） 災害補償・業務外の傷病扶助関係 災害補償及び業務外の傷病扶助に関する事項
（７） 表彰・制裁関係 表彰及び制裁の種類及び程度に関する事項
（８） その他 事業場の労働者すべてに適用されるルールに関する事項なお、就業規則の内容は、法令及び当該事業場において適用される労働協約に反してはなりません。法令又は労働協約に反する就業規則については、所轄労働基準監督署長はそ
の変更を命ずることができます（労基法第９２条）。
３ 就業規則の作成及び変更の手続 労基法は、労働者を１人でも使用する事業場に適用されますが、就業規則については、常時１０人以上の労働者を使用する事業場においては、これを作成しまたは変更する場合に、所轄労働基準監督署長に届け出なければならないとされています（労基法第８９条）。
また、就業規則は、企業単位ではなく事業場単位で作成し、届け出なければなりません。例えば、１企業で２以上の営業所、店舗等を有している場合、企業全体の労働者の数を合計するのではなく、それぞれの営業所、店舗等を１つの事業場としてとらえ、常時使用する労働者が１０人以上の事業場について就業規則を作成する義務が生じます。なお、複数の営業所、店舗等の事業場を有する企業については、営業所、店舗等の就業規則が変更前、変更後ともに本社の就業規則と同一の内容のものである場合に限り、本社所在地を管轄する労働基準監督署長を経由して一括して届け出ることも可能です。
就業規則を作成し、又は変更する場合の所轄労働基準監督署長への届出については、労働者の過半数で組織する労働組合がある場合はその労働組合、過半数で組織する労働組合がない場合は労働者の過半数を代表する者の意見を記し、その者の署名又は記名押印のある書面（意見書）を添付しなければなりません（労基法第９０条）。
この場合の労働者の過半数を代表する者は、①労基法第４１条第２号に規定する監督又は管理の地位にある者でないこと、②就業規則の作成及び変更の際に、使用者から意見を聴取される者を選出することを明らかにして実施する投票、挙手等の方法によって選出された者であることのいずれにも該当する者でなければなりません（労働基準法施行規則（昭和２２年厚生省令第２３号。以下、「労基則」といいます。）第６条の２）。
４ 就業規則の作成又は変更に当たっては、その内容をよく吟味するとともに上記の手続等を遵守しなければなりません。特に、就業規則を労働者にとって不利益に変更する場合には、労働者の代表の意見を十分に聴くとともに、変更の理由及び内容が合理的なものとなるよう慎重に検討することが必要です。就 業 規 則 の 届 出 に つ い て は 電 子 申 請 で も 行 う こ と が 可 能 で す 。 詳 細 は こ ち ら（https://www.mhlw.go.jp/stf/seisakunitsuite/bunya/0000184033.html）をご確認ください。
４ 就業規則の周知
作成した就業規則は、労働者の一人ひとりへの配付、労働者がいつでも見られるように職場の見やすい場所への掲示、備付け、あるいは電子媒体に記録し、それを常時モニター画面等で確認できるようにするといった方法により、労働者に周知しなければなりません（労基法第１０６条第１項）。
就業規則は、作成したり、労働者の代表者から意見を聴取しただけでは効力は発生しないと解されています。就業規則の効力発生時期は、就業規則が何らかの方法によって労働者に周知された時期以降で、就業規則に施行期日が定められているときはその日、就業規則に施行期日が定められていないときは、通常は労働者に周知された日と解されています。
５ モデル就業規則の活用に当たってこのモデル就業規則（以下「本規則」といいます。）は、表紙に記載の時点での関係法令等の規定を踏まえ就業規則の規程例を解説とともに示したものです。本規則はあくまでモデル例であり、就業規則の内容は事業場の実態に合ったものとしなければなりません。したがって、就業規則の作成に当たっては、各事業場で労働時間、賃金などの内容を十分検討するようにしてください。
本規則１０ページ以降にある下線部分（例えば、規程例第１条第１項及び第２条第１項中の「 株式会社」や、第５条第１項中の「 週間以内」などの下線部分）については、法令に従い、各事業場の実情に応じて具体的な名称や数字等を定めてください。
また、規程例の下線部の一部（例えば、１か月単位の変形労働時間制（隔週週休２日制を採用する場合）の規程例第１９条第２項中の「７ 時間１５分」などの部分や、第４３条第２項中の「無給／通常の賃金を支払うこと」の部分）には、あらかじめ数字や文言を記入しているものがありますが、これらは規程例の内容を分かりやすく解説するために便宜的に記入したものですので、これらについても、法令に従い各事業場の実情に応じて具体的な数字等を定めてください。
また、本規則は、主として通常の労働者への適用を想定して作成しています。したがって、パートタイム労働者や有期雇用労働者等を雇用している場合、就業規則の作成に当たっては、本規則の各条項についてパートタイム労働者や有期雇用労働者等への適用の可否について必ず検討し、必要に応じて別個の就業規則を作成してください。
なお、パートタイム労働者や有期雇用労働者に関する事項について就業規則を作成したり、変更する場合には、その事業場において雇用するパートタイム労働者や有期雇用労働者の過半数を代表すると認められる者の意見を聴くように努めなければなりません（短時間労働者及び有期雇用労働者の雇用管理の改善等に関する法律（平成５年法律第７６号。以下「パートタイム・有期雇用労働法」といいます。）第７条）。
６ 目 次
第１章 総則…………………………………………………………１０
第 １条（目的）
第 ２条（適用範囲）
第 ３条（規則の遵守）
第２章 採用、異動等………………………………………………１２
第 ４条（採用手続）
第 ５条（採用時の提出書類）
第 ６条（試用期間）
第 ７条（労働条件の明示）
第 ８条（人事異動）
第 ９条（休職）
第３章 服務規律……………………………………………………１６
第１０条（服務）
第１１条（遵守事項）
第１２条（職場のパワーハラスメントの禁止）
第１３条（セクシュアルハラスメントの禁止）
第１４条（妊娠・出産・育児休業・介護休業等に関するハラスメントの禁止）
第１５条（その他あらゆるハラスメントの禁止）
第１６条（個人情報保護）
第１７条（始業及び終業時刻の記録）
第１８条（遅刻、早退、欠勤等）
第４章 労働時間、休憩及び休日…………………………………２２
[例１] 完全週休２日制を採用する場合の規程例
第１９条（労働時間及び休憩時間）
第２０条（休日）
[例２] １か月単位の変形労働時間制（隔週週休２日制を採用する場合）の規程例
７
第１９条（労働時間及び休憩時間）
第２０条（休日）
[例３] １年単位の変形労働時間制の規程例
第１９条（労働時間及び休憩時間）
第２０条（休日）
第２１条（時間外及び休日労働）
第２２条（勤務間インターバル制度）
[例１] インターバル時間と翌日の所定労働時間が重複する部分を働いたものとみ
なす場合
[例２] インターバル時間と翌日の所定労働時間が重複した時、勤務開始時刻を繰り
下げる場合
第５章 休暇等………………………………………………………３８
第２３条（年次有給休暇）
第２４条（年次有給休暇の時間単位での付与）
第２５条（産前産後の休業）
第２６条（母性健康管理の措置）
第２７条（育児時間及び生理休暇）
第２８条（育児・介護休業、子の看護休暇等）
第２９条（不妊治療休暇）
第３０条（慶弔休暇）
第３１条（病気休暇）
第３２条（裁判員等のための休暇）
第６章 賃金…………………………………………………………４７
第３３条（賃金の構成）
第３４条（基本給）
第３５条（家族手当）
第３６条（通勤手当）
第３７条（役付手当）
第３８条（技能・資格手当）
第３９条（精勤手当）
第４０条（割増賃金）
第４１条（１年単位の変形労働時間制に関する賃金の精算）
８
第４２条（代替休暇）
第４３条（休暇等の賃金）
第４４条（臨時休業の賃金）
第４５条（欠勤等の扱い）
第４６条（賃金の計算期間及び支払日）
第４７条（賃金の支払と控除）
第４８条（賃金の非常時払い）
第４９条（昇給）
第５０条（賞与）
第７章 定年、退職及び解雇………………………………………６６
第５１条（定年等）
[例１] 定年を満６５歳とする例
[例２] 定年を満６０歳とし、その後希望者を再雇用する例
［例３］ 定年を満６０歳とし、その後希望者を継続雇用する例（満６５歳以降は対
象者基準あり）
［例４］ 定年を満６５歳とし、その後希望者の意向を踏まえて継続雇用または業務
委託契約を締結する例（ともに対象者基準あり）
第５２条（退職）
第５３条（解雇）
第８章 退職金………………………………………………………７４
第５４条（退職金の支給）
第５５条（退職金の額）
第５６条（退職金の支払方法及び支払時期）
第９章 無期労働契約への転換……………………………………７６
第５７条（無期労働契約への転換）
第１０章 安全衛生及び災害補償…………………………………７８
第５８条（遵守事項）
第５９条（健康診断）
９
第６０条（長時間労働者に対する面接指導）
第６１条（ストレスチェック）
第６２条（健康管理上の個人情報の取扱い）
第６３条（安全衛生教育）
第６４条（災害補償）
第１１章 職業訓練…………………………………………………８４
第６５条（教育訓練）
第１２章 表彰及び制裁……………………………………………８５
第６６条（表彰）
第６７条（懲戒の種類）
第６８条（懲戒の事由）
第１３章 公益通報者保護…………………………………………８９
第６９条（公益通報者の保護）
第１４章 副業・兼業………………………………………………９０
第７０条（副業・兼業）
- 10 -
第１章 総則
総則には、一般的に就業規則の作成の目的や適用範囲等を規定します。
（目的）
第１条 この就業規則（以下「規則」という。
）は、労働基準法（以下「労基法」とい
う。
）第８９条に基づき、 株式会社の労働者の就業に関する事項を定めるもの
である。
２ この規則に定めた事項のほか、就業に関する事項については、労基法その他の法令
の定めによる。
【第１条 目的】
１ この就業規則規程例（以下「本規程例」といいます。）では、労働者の就業に関する事項を定めていますが、その前提にある法令上の基準は、労基法等関係法令に定められています。
２ 本規程例に労働者の就業に関するすべての事項が定められているわけではありません。本規程例に定めがない事項については、労基法等関係法令の規定によることになります。
３ 就業規則で定める基準に達しない労働条件を定める労働契約は、その部分については無効となります。この場合において、無効となった部分は、就業規則で定める基準によることになります（労働契約法（平成１９年法律第１２８号。以下「契約法」といいま
す。）第１２条）。また、就業規則は法令又は事業場に適用される労働協約に反してはなりません（労基法第９２条）。
（適用範囲）第２条 この規則は、 株式会社の労働者に適用する。
２ パートタイム労働者の就業に関する事項については、別に定めるところによる。
３ 前項については、別に定める規則に定めのない事項は、この規則を適用する。
【第２条 適用範囲】
１ 就業規則は、すべての労働者について作成する必要があります。しかし、就業規則は、必ずしもすべての労働者について同一のものでなければならないわけではありません。同一の事業場であっても、通常の労働者と勤務態様の異なるパートタイム労働者等については、一定の事項について特別の規定を設けたり、別の就業規則を定めることができます。パートタイム労働者等について、規程の一部を適用除外とする場合や全面的に適用除外とする場合には、就業規則本体にその旨明記し、パートタイム労働者等に適用される規定を設けたり、別の就業規則を作成しなければなりません。本規程例では、パートタイム労働者の就業に関する事項について、就業規則本体とは別に定める形式をとっています。パートタイム・有期雇用労働者の就業規則の規程例等は、こちら（https://www.mhlw.go.jp/stf/seisakunitsuite/bunya/0000046152.html）。
２ 働き方改革を推進するための関係法律の整備に関する法律（平成３０年法律第７１号）による改正により、２０２０年４月（中小企業におけるパートタイム労働者、有期雇用労働者については２０２１年４月）より、パートタイム労働者や有期雇用労働者、派遣労働者の待遇について、職務内容、職務内容・配置の変更範囲等を考慮して、通常の労働者との間で不合理な待遇差を設けることは禁止されます（パートタイム・有期雇用労働法第８条、第９条及び労働者派遣事業の適正な運営の確保及び派遣労働者の保護等に関する法律（昭和６０年法律第８８号）第３０条の３）。これらの法律では、賃金だけでなく、福利厚生、休暇などすべての待遇が対象とされています。パートタイム労働者・有期雇用労働者と通常の労働者との間で、賃金等について取扱いに違いがある場合、パートタイム・有期雇用労働者から求められたときは、相違の内容及び理由について説明する必要があります。（パートタイム・有期雇用労働法第１４条第２項）。
（規則の遵守） 第３条 会社は、この規則に定める労働条件により、労働者に就業させる義務を負う。また、労働者は、この規則を遵守しなければならない。
【第３条 規則の遵守】 労基法第２条において、労働者及び使用者は、就業規則等を遵守し、誠実に各々その義務を履行しなければならないと規定されています。
第２章 採用、異動等 採用、異動等については、一般的に採用に際しての手続に関する事項、試用期間、労働条件の明示、人事異動、休職に関すること等を定めます。
（採用手続） 第４条 会社は、入社を希望する者の中から選考試験を行い、これに合格した者を採用する。
【第４条 採用手続】
１ 会社は、労働者の採用に当たり、男女かかわりなく均等な機会を与えなければなりません（雇用の分野における男女の均等な機会及び待遇の確保等に関する法律（昭和４７年法律第１１３号。以下「均等法」といいます。）第５条）。
２ 合理的な理由がない場合に、労働者の採用において身長・体重・体力を要件とすること、転居を伴う転勤に応じることを要件とすること等は、間接差別として禁止されています（均等法第７条）。
（採用時の提出書類） 第５条 労働者として採用された者は、採用された日から 週間以内に次の書類を提出しなければならない。 ① 住民票記載事項証明書 ② 自動車運転免許証の写し（ただし、自動車運転免許証を有する場合に限る。 ） ③ 資格証明書の写し（ただし、何らかの資格証明書を有する場合に限る。 ） ④ その他会社が指定するもの
２ 前項の定めにより提出した書類の記載事項に変更を生じたときは、速やかに書面で会社に変更事項を届け出なければならない。
【第５条 採用時の提出書類】 会社は、労働者の年齢、現住所を確認するに当たり、労働者から戸籍謄本（抄本）や住民票の写しを提出させることは適切ではありません。住民票記載事項の証明書により処理することが適切です。また、提出させる書類については、その提出目的を労働者に説明し、明らかにしてください。
（試用期間） 第６条 労働者として新たに採用した者については、採用した日から か月間を試用期間とする。
２ 前項について、会社が特に認めたときは、試用期間を短縮し、又は設けないことがある。
３ 試用期間中に労働者として不適格と認めた者は、解雇することがある。ただし、入社後１４日を経過した者については、第５３条第２項に定める手続によって行う。
４ 試用期間は、勤続年数に通算する。
【第６条 試用期間】 １ 試用期間を設ける場合にその期間の長さに関する定めは労基法上ありませんが、労働者の地位を不安定にすることから、あまりに長い期間を試用期間とすることは好ましくありません。
２ 試用期間中の解雇については、最初の１４日間以内であれば即時に解雇することができますが、試用期間中の者も１４日を超えて雇用した後に解雇する場合には、原則として３０日以上前に予告をしなければなりません。予告をしない場合には、平均賃金の３０日分以上の解雇予告手当を支払うことが必要となります（労基法第２０条、第２１条）。
（労働条件の明示） 第７条 会社は、労働者を採用するとき、採用時の賃金、就業場所、従事する業務、労働時間、休日、その他の労働条件を記した労働条件通知書及びこの規則を交付して労働条件を明示するものとする。
【第７条 労働条件の明示】
１ 労働者を雇い入れるに際し、労働者に賃金、労働時間、その他の労働条件を明示することが必要です。特に、労働条件を明示するに当たり、次の（１）から（６）までの項目（昇給に関する事項を除く）については、原則書面の交付により明示する必要があります（労基法第１５条、労基則第５条）。 （１） 労働契約の期間に関する事項 （２） 期間の定めのある労働契約を更新する場合の基準に関する事項（期間の定めのある労働契約を更新する場合に限る） （３） 就業の場所及び従事すべき業務に関する事項 （４） 始業及び終業の時刻、所定労働時間を超える労働の有無、休憩時間、休日、休 暇並びに交替制により就業させる場合における就業時転換に関する事項 （５） 賃金（退職手当及び臨時に支払われる賃金等を除く。）の決定、計算及び支払の方法、賃金の締切り及び支払の時期並びに昇給に関する事項
労働者が以下のいずれかの方法を希望した場合には、当該方法により労働条件の明示を行うことができます。 ・ ファクシミリを利用して送信する方法 ・ 電子メールその他のその受信をする者を特定して情報を伝達するために用いられる電気通信（有線、無線その他の電磁的方法により、符号、音響、又は影像を送り、伝え、又は受けることをいう。以下「電子メール等」という。）の送信の方法（当該労働者が当該電子メール等の記録を出力することにより書面を作成することができるものに限る。）※「電子メール等」には以下が含まれます。 ① E メール、Yahoo!メールや Gmail 等のウェブメールサービス ② ＋メッセージ等の RCS（リッチ・コミュニケーション・サービス）や、SMS（ショ ート・メール・サービス） ③ LINE や Facebook 等の SNS メッセージ機能 ただし、ブログやホームページへの書き込みのように、特定の個人がその入力する情報を電気通信を利用して第三者に閲覧させることに付随して、当該第三者が当該個人に対し情報を伝達することができる機能が提供されるものについては、「その受信する者を特定して情報を伝達するために用いられる電気通信」には含まれないため、この方法により労働条件の明示を行うことはできません。さらに、パートタイム・有期雇用労働者については、雇入れに際して、昇給、退職手当、賞与の有無、相談窓口についても文書の交付等により明示しなければなりません（パートタイム・有期雇用労働法第６条第１項）。
２ また、採用内定により労働契約が成立していると解される場合がありますが、この場合には、採用内定に際して、内定者に労働条件を書面で明示する必要があります。（人事異動）第８条 会社は、業務上必要がある場合に、労働者に対して就業する場所及び従事する業務の変更を命ずることがある。
２ 会社は、業務上必要がある場合に、労働者を在籍のまま関係会社へ出向させることがある。
３ 前２項の場合、労働者は正当な理由なくこれを拒むことはできない。（６） 退職に関する事項（解雇の事由を含む。）
【第８条 人事異動】
１ 労働者を採用した後、会社が業務上の理由から就業場所や従事する業務を変更することは、変更がない旨の特別な合意等がない限り可能です。しかしながら、労働者の意に沿わない就業場所等の変更を命じた場合、トラブルが生じ得ますので、本規則のように就業規則に明記しておくことが望ましいと言えます。もちろん、労働者の同意を得るようにすることが大切であることは言うまでもありません。なお、労働者の就業場所を変更しようとする場合には、労働者の育児や介護の状況に配慮しなければなりません（育児休業、介護休業等育児又は家族介護を行う労働者の福祉に関する法律（平成３年法律第７６号。以下「育児・介護休業法」といいます。）第２６条）
２ また、他の会社へ出向させることが想定される場合、出向に関する規定を設けておく必要があります。
（休職） 第９条 労働者が、次のいずれかに該当するときは、所定の期間休職とする。① 業務外の傷病による欠勤が か月を超え、なお療養を継続する必要があるため勤務できないとき 年以内② 前号のほか、特別な事情があり休職させることが適当と認められるとき必要な期間
２ 休職期間中に休職事由が消滅したときは、原則として元の職務に復帰させる。ただし、元の職務に復帰させることが困難又は不適当な場合には、他の職務に就かせることがある。
３ 第１項第１号により休職し、休職期間が満了してもなお傷病が治癒せず就業が困難な場合は、休職期間の満了をもって退職とする。
【第９条 休職】
１ 休職とは、業務外での疾病等主に労働者側の個人的事情により相当長期間にわたり就労を期待し得ない場合に、労働者としての身分を保有したまま一定期間就労義務を免除する特別な扱いをいいます。なお、本条第１項第２号の「特別な事情」には、公職への就任や刑事事件で起訴された場合等が当たります。
２ 休職期間中に休職事由がなくなった場合は、当然に休職が解除され復職となります。
３ 休職の定義、休職期間の制限、復職等については、労基法に定めはありません。
第３章 服務規律 （服務） 第１０条 労働者は、職務上の責任を自覚し、誠実に職務を遂行するとともに、会社の指示命令に従い、職務能率の向上及び職場秩序の維持に努めなければならない。（遵守事項）
第１１条 労働者は、以下の事項を守らなければならない。 ① 許可なく職務以外の目的で会社の施設、物品等を使用しないこと。 ② 職務に関連して自己の利益を図り、又は他より不当に金品を借用し、若しくは贈 与を受ける等不正な行為を行わないこと。 ③ 勤務中は職務に専念し、正当な理由なく勤務場所を離れないこと。 ④ 会社の名誉や信用を損なう行為をしないこと。 ⑤ 在職中及び退職後においても、業務上知り得た会社、取引先等の機密を漏洩しな いこと。 ⑥ 酒気を帯びて就業しないこと。 ⑦ その他労働者としてふさわしくない行為をしないこと。
【第１０条 服務】
【第１１条 遵守事項】 服務規律及び遵守事項については、就業規則に必ず定めなければならない事項ではありませんが、職場の秩序維持に大きな役割を果たすことから、会社にとって労働者に遵守させたい事項を定めてください。
【第１２条 職場のパワーハラスメントの禁止】 職場におけるパワーハラスメントを防止するために、事業主は、雇用管理上必要な措
置を講じなければならないこととされています（労働施策の総合的な推進並びに労働者
の雇用の安定及び職業生活の充実等に関する法律（昭和４１年法律第１３２号。以下「労働施策総合推進法」といいます。）第３０条の２）。
（職場のパワーハラスメントの禁止） 第１２条 職務上の地位や人間関係などの職場内の優越的な関係を背景とした、業務上必要かつ相当な範囲を超えた言動により、他の労働者の就業環境を害するようなことをしてはならない。
（セクシュアルハラスメントの禁止） 第１３条 性的言動により、他の労働者に不利益や不快感を与えたり、就業環境を害するようなことをしてはならない。
【第１３条 セクシュアルハラスメントの禁止】 職場におけるセクシュアルハラスメントを防止するため、事業主は、雇用管理上必要な措置を講じなければならないこととされています（均等法第１１条）。（妊娠・出産・育児休業・介護休業等に関するハラスメントの禁止）
第１４条 妊娠・出産等に関する言動及び妊娠・出産・育児・介護等に関する制度又は措置の利用に関する言動により、他の労働者の就業環境を害するようなことをしてはならない。
【第１４条 妊娠・出産・育児休業・介護休業等に関するハラスメントの禁止】 職場における妊娠・出産・育児休業・介護休業等に関するハラスメントを防止するため、事業主は、雇用管理上必要な措置を講じなければならないこととされています（均等法第１１条の３、育児・介護休業法第２５条）。
（その他あらゆるハラスメントの禁止） 第１５条 第１２条から前条までに規定するもののほか、性的指向・性自認に関する言動によるものなど職場におけるあらゆるハラスメントにより、他の労働者の就業環境を害するようなことをしてはならない。
【第１５条 その他あらゆるハラスメントの禁止】 恋愛感情又は性的感情の対象となる性別についての指向のことを「性的指向」、自己の性別についての認識のことを「性自認」といいます。性的指向や性自認への理解を深め、差別的言動や嫌がらせ（ハラスメント）が起こらないようにすることが重要です。
（個人情報保護） 第１６条 労働者は、会社及び取引先等に関する情報の管理に十分注意を払うとともに、自らの業務に関係のない情報を不当に取得してはならない。
２ 労働者は、職場又は職種を異動あるいは退職するに際して、自らが管理していた会社及び取引先等に関するデータ・情報書類等を速やかに返却しなければならない。
【第１６条 個人情報保護】 個人情報の保護に関する法律（平成１５年法律第５７号）の全面施行により、使用者に個人情報の適正な管理に関する対策が義務付けられています。
（始業及び終業時刻の記録） 第１７条 労働者は、始業及び終業時にタイムカードを自ら打刻し、始業及び終業の時刻を記録しなければならない。
【第１７条 始業及び終業時刻の記録】 労働時間の管理については、「労働時間の適正な把握のために使用者が講ずべき措置に関するガイドライン」（平成２９年１月２０日策定）で、使用者が講ずべき措置が具体的に示されています。使用者は、このガイドラインを遵守し、労働時間を適正に把握する等適切な時間管理を行ってください。なお、後述するように平成３１年４月から労働安全衛生法（昭和４７年法律第５７号。以下「安衛法」といいます。）第６６条の８の３の規定に基づき事業者は、面接指導を実施するため、タイムカードによる記録、パーソナルコンピュータ等の電子計算機の使用時間（ログインからログアウトまでの時間）の記録等の客観的な方法その他の適切な方法により、労働者の労働時間の状況を把握しなければなりません（本規則第６０条参照）。
（参考） 「労働時間の適正な把握のために使用者が講ずべき措置に関するガイドライン（抜粋）」
１． 趣旨 労働基準法においては、労働時間、休日、深夜業等について規定を設けていることから、使用者は、労働時間を適正に把握するなど労働時間を適切に管理する責務を有している。
２． 労働時間の考え方 労働時間とは、使用者の指揮命令下に置かれている時間のことをいい、使用者の明示又は黙示の指示により労働者が業務に従事する時間は労働時間に当たる。そのため、次のアからウのような時間は、労働時間として扱わなければならないこと。
（ア） 使用者の指示により、就業を命じられた業務に必要な準備行為（着用を義務付けられた所定の服装への着替え等）や業務終了後の業務に関連した後始末（清掃等）を事業場内において行った時間
（イ） 使用者の指示があった場合には即時に業務に従事することを求められており、労働から離れることが保障されていない状態で待機等している時間（いわゆる「手待時間」）
（ウ） 参加することが業務上義務づけられている研修・教育訓練の受講や、使用者の指示により業務に必要な学習等を行っていた時間
３． 始業・終業時刻の確認及び記録 使用者は、労働時間を適正に管理するため、労働者の労働日ごとの始業・終業時刻を確認し、これを記録すること。
４． 始業・終業時刻の確認及び記録の原則的な方法 使用者が始業・終業時刻を確認し、記録する方法としては、原則として次のいずれかの方法によること。
（ア） 使用者が、自ら現認することによりこれを確認し、記録すること。
（イ） タイムカード、ＩＣカード、パソコンの使用時間の記録等の客観的な記録を基礎として確認し、記録すること。
５． 自己申告制により始業・終業時刻の確認及び記録を行う場合の措置 ３の方法によることなく、自己申告制により行わざるを得ない場合、使用者は次の措置を講ずること。
（ア） 自己申告制の対象となる労働者に対して、本ガイドラインを踏まえ、労働時間の実態を正しく記録し、適正に自己申告を行うことなどについて十分な説明を行うこと。
（イ） 実際に労働時間を管理する者に対して、自己申告制の適正な運用を含め、本ガイドラインに従い講ずべき措置について十分な説明を行うこと。
（ウ） 自己申告により把握した労働時間が実際の労働時間と合致しているか否かについて、必要に応じて実態調査を実施し、所要の労働時間の補正をすること。
特に、入退場記録やパソコンの使用時間の記録など、事業場内にいた時間の分かるデータを有している場合に、労働者からの自己申告により把握した労働時間と当該データで分かった事業場内にいた時間との間に著しい乖離が生じているときには、実態調査を実施し、所要の労働時間の補正をすること。
（エ） 自己申告した労働時間を超えて事業場内にいる時間について、その理由等を労働者に報告させる場合には、当該報告が適正に行われているかについて確認すること。その際、休憩や自主的な研修、教育訓練、学習等であるため労働時間ではないと報告されていても、実際には、使用者の指示により業務に従事しているなど使用者の指揮命令下に置かれていたと認められる時間については、労働時間として扱わなければならないこと。
（オ） 自己申告制は、労働者による適正な申告を前提として成り立つものである。このため、使用者は、労働者が自己申告できる時間外労働の時間数に上限を設け、上限を超える申告を認めない等、労働者による労働時間の適正な申告を阻害する措置を講じてはならないこと。また、時間外労働時間の削減のための社内通達や時間外労働手当の定額払等労働時間に係る事業場の措置が、労働者の労働時間の適正な申告を阻害する要因となっていないかについて確認するとともに、当該要因となっている場合においては、改善のための措置を講ずること。
さらに、労働基準法の定める法定労働時間や時間外労働に関する労使協定（いわゆる 36 協定）により延長することができる時間数を遵守することは当然であるが、実際には延長することができる時間数を超えて労働しているにもかかわらず、記録上これを守っているようにすることが、実際に労働時間を管理する者や労働者等において、慣習的に行われていないかについても確認すること
６．賃金台帳の適正な調製 使用者は、労働基準法第１０８条及び同法施行規則第５４条により、労働者ごとに、労働日数、労働時間数、休日労働時間数、時間外労働時間数、深夜労働時間数といった事項を適正に記入しなければならないこと。また、賃金台帳にこれらの事項を記入していない場合や、故意に賃金台帳に虚偽の労働時間数を記入した場合は、同法第１２０条に基づき、３０万円以下の罰金に処されること。
７．労働時間の記録に関する書類の保存 使用者は、労働者名簿、賃金台帳のみならず、出勤簿やタイムカード等の労働時間の記録に関する書類について、労働基準法第１０９条に基づき、３年間保存しなければならないこと。
（遅刻、早退、欠勤等） 第１８条 労働者は遅刻、早退若しくは欠勤をし、又は勤務時間中に私用で事業場から外出する際は、事前に に対し申し出るとともに、承認を受けなければならない。ただし、やむを得ない理由で事前に申し出ることができなかった場合は、事後に速やかに届出をし、承認を得なければならない。
２ 前項の場合は、第４５条に定めるところにより、原則として不就労分に対応する賃金は控除する。
３ 傷病のため継続して 日以上欠勤するときは、医師の診断書を提出しなければならない。
【第１８条 遅刻、早退、欠勤等】
１ 本規程例では労働者が遅刻、早退若しくは欠勤等をする場合、事前の申出と会社の承認を得ることとしていますが、どのような手続を規定するかは各事業場で決めることで
す。しかし、こうした手続を取ることは会社の秩序を維持する上でも重要なこととなりますので、明確に定めてください。
２ 欠勤何日以上で医師の診断書を提出させるかは、各事業場で決めることです。
第４章 労働時間、休憩及び休日 １ 労働時間、休憩及び休日に関することは、就業規則の絶対的必要記載事項に当たります。
２ 労基法第３２条第１項において、１週間の労働時間の上限は４０時間と定められています。ただし、特例措置として、商業（労基法別表第１第８号）、映画の製作の事業を除く映画・演劇業（同第１０号）、保健衛生業（同第１３号）、接客娯楽業（同第１４号）の事業であって、労働者数１０人未満の事業場（以下「特例措置対象事業場」といいます。）は、１週４４時間まで働かせることが認められています（労基法第４０条、労基則第２５条の２）。また、労基法第３２条第２項において、１日の労働時間の上限は８時間と定められています。
３ 休憩時間については、１日の労働時間が６時間を超える場合には少なくとも４５分、８時間を超える場合には少なくとも１時間の休憩時間を与えなければなりません（労基法第３４条）。
４ 休日については、毎週少なくとも１回又は４週間を通じ４日以上与えなければなりません（労基法第３５条）。
５ 上記２から４までの労基法の規定に適合する労働条件とするためには、①週休２日制とする、②週休１日制で１日の所定労働時間を短く設定する、③変形労働時間制（１か月単位、１年単位等）を導入する等の方法がありますので、それぞれの事業場の実情に応じて、下記の規程例を参考に就業規則を作成してください。［例１］ 完全週休２日制を採用する場合の規程例１日の労働時間を８時間とし、完全週休２日制を採用する場合の規程例です。
(労働時間及び休憩時間) 第１９条 労働時間は、１週間については４０時間、１日については８時間とする。
２ 始業・終業の時刻及び休憩時間は、次のとおりとする。ただし、業務の都合その他やむを得ない事情により、これらを繰り上げ、又は繰り下げることがある。この場合、前日までに労働者に通知する。
① 一般勤務始業・終業時刻 休憩時間始業 午前 時 分時 分から 時 分まで終業 午後 時 分
② 交替勤務（イ）１番（日勤）始業・終業時刻 休憩時間始業 午前 時 分時 分から 時 分まで終業 午後 時 分
（ロ）２番（準夜勤）始業・終業時刻 休憩時間始業 午前 時 分時 分から 時 分まで終業 午後 時 分
（ハ）３番（夜勤）始業・終業時刻 休憩時間始業 午前 時 分時 分から 時 分まで終業 午後 時 分
３ 交替勤務における各労働者の勤務は、別に定めるシフト表により、前月の 日までに各労働者に通知する。
４ 交替勤務における就業番は原則として 日ごとに 番を 番に、番を 番に、 番を 番に転換する。
５ 一般勤務から交替勤務へ、交替勤務から一般勤務への勤務形態の変更は、原則として休日又は非番明けに行うものとし、前月の 日前までに が労働者に通知する。
【第１９条 労働時間及び休憩時間】 １ 始業及び終業の時刻、休憩時間は、就業規則に必ず定めておかなければなりません。また、交替勤務をとる場合は、勤務形態ごとの始業・終業時刻及び休憩時間を規定するとともに、就業番の転換についても就業規則に規定してください。
２ 休憩は、原則として事業場すべての労働者に一斉に与えなければなりませんが、本規程例のように交替勤務を採用する等一斉に与えることが困難な場合には、労働者代表との書面による協定（以下「労使協定」といいます。）を結ぶことにより交替で与えることができます（労基法第３４条第２項）。この場合、一斉に休憩を与えない労働者の範囲及び当該労働者に対する休憩の与え方について、労使協定で定めなければなりません（労基則第１５条）。また、一斉休憩付与に対する例外として、労基法第４０条に基づき、労基則第３１条において、運輸交通業（労基法別表第１第４号）、商業（同第８号）、金融・広告業（同第９号）、映画・演劇業（同第１０号）、通信業（同第１１号）、保健衛生業（同第１３号）、接客娯楽業（同第１４号）及び官公署の事業について、一斉に休憩を与えなくてもよい旨が定められています。労使協定の労働者代表については、本規程例第２１条の解説を参照してください。
３ 休憩時間は、労働者に自由に利用させなければなりません。使用者の指示があった場合には即時に業務に従事することを求められており、労働から離れることが保障されていない状態で待機等している時間（いわゆる「手待ち時間」）については労働時間に当たり休憩時間ではありませんので注意してください。（休日）
第２０条 休日は、次のとおりとする。 ① 土曜日及び日曜日 ② 国民の祝日（日曜日と重なったときは翌日） ③ 年末年始（１２月 日～１月 日） ④ 夏季休日（ 月 日～ 月 日） ⑤ その他会社が指定する日
２ 業務の都合により会社が必要と認める場合は、あらかじめ前項の休日を他の日と振り替えることがある。
【第２０条 休日】 １ 労基法では何曜日を休日とするかあるいは国民の祝日を休日とするかについて規定していません。１週間の中で何曜日を休日としても、また、週によって異なる曜日を休日
としても差し支えありません。さらに、勤務の実態に合わせて、労働者ごとに異なる日に交替で休日を与えることもできます。
２ 休日は、原則として暦日（午前０時から午後１２時までの継続２４時間をいう。）で与えなければなりません。しかし、番方編成による交替制（８時間３交替勤務のような場合をいう。）を導入するような場合、以下の要件を満たせば休日は暦日ではなく、継続した２４時間を与えれば差し支えないとされています（昭和６３年３月１４日付け基発１５０号）。
（イ）番方編成による交替制によることが就業規則等により定められており、制度として運用されていること。
（ロ）各番方の交替が規則的に定められているものであって、勤務割表等によりその都度設定されるものではないこと。
３ 本条第２項において定めている、いわゆる「振替休日」とは、例えば業務の都合によって所定休日である日曜日に勤務させなければならない場合に、当該日曜日を勤務日に変更し、その代わり勤務日である例えば月曜日を休日とするように、所定の休日とあらかじめ他の勤務日と振り替えることをいいます。また、｢代休｣とは、休日に休日労働を行わせた場合に、その代わりに以後の特定の勤務日又は労働者の希望する任意の勤務日の労働義務を免除し、休みを与える制度のことをいいます。振替休日と代休の労基法上での取扱いの違いは次のとおりです。「労働基準法上の振替休日と代休の取扱いの違い」① 振替休日は、あらかじめ定められた法定休日を他の日に振り替えることですから、振替前の休日に勤務しても通常の勤務と同じです。したがって、休日労働に対する割増賃金の問題は発生しませんが、振り替えた休日が週をまたがった場合、振替勤務したことにより、当該週の実労働時間が週の法定労働時間を超える場合があります。その場合は時間外労働に対する割増賃金の支払が必要となります。
その一方で、代休は、定められた法定休日に休日労働を行わせた場合ですから、その後に代休を与えても休日労働をさせたことが帳消しにされるものではありませんので、休日労働に対する割増賃金を支払う必要があります。
② 休日は労働者の労働義務のない日ですから、これを振り替える場合は、以下に示す措置が必要となります。 ア 就業規則に振替休日の規程を置くこと。 イ 振替休日は特定すること。 ウ 振替休日は４週４日の休日が確保される範囲のできるだけ近接した日とすること。 エ 振替は前日までに通知すること。
〔例２〕１か月単位の変形労働時間制（隔週週休２日制を採用する場合）の規程例 〔例２〕は、１か月単位の変形労働時間制（変形期間は２週間）を活用しつつ、隔週での週休２日制で、毎日の所定労働時間を７時間１５分とすることにより、週４０時間労働制を実施する場合の規程例です。
（労働時間及び休憩時間） 第１９条 １週間の所定労働時間は、 年 月 日を起算日として、２週間ごとに平均して、１週間当たり４０時間とする。
２ １日の所定労働時間は、７ 時間１５分とする。
３ 始業・終業の時刻及び休憩時間は、次のとおりとする。ただし、業務の都合その他やむを得ない事情により、これらを繰り上げ、又は繰り下げることがある。この場合において業務の都合によるときは、 が前日までに通知する。 始業・終業時刻 休憩時間 始業 午前 時 分 時 分から 時 分まで 終業 午後 時 分
（休日） 第２０条 休日は、次のとおりとする。① 日曜日 ② 年 月 日を起算日とする２週間ごとの第２ 土曜日 ③ 国民の祝日（日曜日と重なったときは翌日） ④ 年末年始（１２月 日～１月 日） ⑤ 夏季休日（ 月 日～ 月 日） ⑥ その他会社が指定する日
２ 業務の都合により会社が必要と認める場合は、あらかじめ前項の休日を他の日と振り替えることがある。
【第１９条 労働時間及び休憩時間】
【第２０条 休日】 １ １か月単位の変形労働時間制とは、労使協定又は就業規則等により、１か月以内の一定期間を平均して１週間当たりの労働時間が４０時間を超えない定めをした場合においては、その定めにより、特定された日又は特定された週に１日８時間又は１週４０時間を超えて労働させることができるという制度です（労基法第３２条の２）。この場合の労使協定は、所轄の労働基準監督署長に届け出ることが必要です。労使協定の労働者代表の選出方法等ついては、本規程例第２１条の解説を参照してください。
２ 本規程例は、１日の所定労働時間を固定していますが、業務の都合等によって日々の所定労働時間を変えることもできます。この場合も、一定期間を平均して１週当たりの労働時間が４０時間を超えないようにしなければなりません。
３ １か月単位の変形労働時間制を採用する場合には、就業規則等において変形期間の起算日や各日の始業・終業の時刻及び変形期間内の各日・各週の労働時間を明確にしておくことが必要です。
４ 以下とおり、 〔例２〕の場合は、２週間の所定労働時間は合計７９時間４５分となるため、１週間当たりの平均所定労働時間は３９時間５３分となり、週４０時間以下を満たすこととなります。４３時間３０分 ３６時間１５分 ７ 時 間 15 分 〃 〃 〃 〃 〃 休 日 7 時 間 15 分 〃 〃 〃 〃 休 日 休 日 １ 日 目 月 ２ 日 目 火 ３ 日 目 水 ４ 日 目 木 ５ 日 目 金 ６ 日 目 土 ７ 日 目 日 ８ 日 目 月 ９ 日 目 火 10 日 目 水 11 日 目 木 12 日 目 金 13 日 目 土 14 日 目 日 なお、 〔例２〕 の規程例第１９条では、 ２週間ごとの第２土曜日を休日としていますが、 国民の祝日等を休日とする場合、国民の祝日等がある週の土曜日（又は日曜日）を出勤 日としても週休２日制となります。この場合、規程例第１９条に「ただし、第２号の期 間に第３号の休日が含まれる場合には、その期間の第２土曜日は出勤日とする。」といった文言を追記する必要があります。
【参考】 「１か月単位の変形労働時間制における所定労働時間の定め方」 １か月単位の変形労働時間制については、１か月以内の一定期間（変形期間）を平均して１週間当たりの労働時間が週の法定労働時間（４０時間）を超えない範囲で、就業規則等に各日、各週の所定労働時間を具体的に定めなければなりません。この場合、変形期間における所定労働時間の合計は次の式によって計算された時間の範囲内で設定します。１週間の法定労働時間(４０時間) ×７変形期間の暦日数この式によって変形期間が１か月の場合の所定労働時間の総枠を計算すると、次の表のとおりとなります。
項目 １か月の暦日数 各変形期間に対応する所定労働時間の総枠 法定労働時間が４０時間の場合 法定労働時間が４４時間の場合３１日の場合 ３０日の場合 ２９日の場合 ２８日の場合 １７７．１時間 １７１．４時間 １６５．７時間 １６０．０時間 １９４．８時間 １８８．５時間 １８２．２時間 １７６．０時間 （注）小数点第２位以下を省略。
また、 １か月単位の変形労働時間制における週休２日制の形態別の週所定労働時間は、 次の表のとおりとなります。 週休２日制の 形態 １日の 所定労働時間等 日曜日及び 月２日土曜休日 日曜日及び 月３日土曜休日 日曜日及び 月４日土曜休日 １ 月 の 日 数 28 日 30 日 31 日 28 日 30 日 31 日 28 日 30 日 31 日 １月の休日数 6 日 6 日 6 日 7 日 7 日 7 日 8 日 8 日 8 日 勤 務 日 数 22 日 24 日 25 日 21 日 23 日 24 日 20 日 22 日 23 日 １日８：００ 44:00 44:48 45:10 42:00 42:56 43:22 40:00 41:04 41:33 １日７：５０ 43:05 43:52 44:13 41:07 42:03 42:27 39:10 40:13 40:41 １日７：４５ 42:38 43:24 43:45 40:41 41:36 42:00 38:45 39:47 40:15 １日７：４０ 42:10 42:56 43:17 40:15 41:09 41:33 38:20 39:22 39:49 １日７：３０ 41:15 42:00 42:21 39:23 40:15 40:39 37:30 38:30 38:57 １日７：２０ 40:20 41:04 41:24 38:30 39:22 39:45 36:40 37:39 38:05 １日７：１５ 39:53 40:36 40:56 38:04 38:55 39:18 36:15 37:13 37:39 １日７：１０ 39:25 40:08 40:28 37:37 38:28 38:50 35:50 36:47 37:13 １日７：００ 38:30 39:12 39:31 36:45 37:34 37:56 35:00 35:56 36:22 （注） の場合は１か月を平均し１週間の労働時間が４０時間を超えるため、４０
時間以下となるよう特定の日の労働時間を少なくする等の調整をする必要があります。
〔例３〕１年単位の変形労働時間制の規程例 （労働時間及び休憩時間） 第１９条 労働者代表と１年単位の変形労働時間制に関する労使協定を締結した場合、 当該協定の適用を受ける労働者について、１週間の所定労働時間は、対象期間を平均 して１週間当たり４０時間とする。 ２ １年単位の変形労働時間制を適用しない労働者について、１週間の所定労働時間は ４０時間、１日の所定労働時間は８時間とする。 ３ １日の始業・終業の時刻、休憩時間は次のとおりとする。 ① 通常期間 始業・終業時刻 休憩時間 始業 午前 時 分 時 分から 時 分まで 終業 午後 時 分 ② 特定期間（１年単位の変形労働時間制に関する労使協定で定める特定の期間を いう。 ） 始業・終業時刻 休憩時間 始業 午前 時 分 時 分から 時 分まで 終業 午後 時 分 ③ １年単位の変形労働時間制を適用しない労働者の始業・終業の時刻、休憩時間 は次のとおりとする。 始業・終業時刻 休憩時間 始業 午前 時 分 時 分から 時 分まで 終業 午後 時 分
（休日） 第２０条 １年単位の変形労働時間制の適用を受ける労働者の休日については、１年単位の変形労働時間制に関する労使協定の定めるところにより、対象期間の初日を起算日とする１週間ごとに１日以上、１年間に 日以上となるように指定する。その場合、年間休日カレンダーに定め、対象期間の初日の３０日前までに各労働者に通知する。
２ １年単位の変形労働時間制を適用しない労働者の休日については、以下のとおり指定し、月間休日カレンダーに定め、対象期間の初日の３０日前までに各労働者に通知する。 ① 日曜日（前条第３号の特定期間を除く。） ② 国民の祝日（日曜日と重なったときは翌日） ③ 年末年始（１２月 日～１月 日） ④ 夏季休日（ 月 日～ 月 日） ⑤ その他会社が指定する日
"""

ダウンロードしたテキストファイルはページ幅で折り返されているので、機械学習には適さない状態です。

ここでは45分かけてちまちま手動で1分節1行になるように行を結合して、286行まで編集しました。45分かかって編集しても、まだ全体の12%...

In [ ]:
from io import StringIO
model_df = pd.read_csv(StringIO(model_code))
model_df

,１ モデル就業規則
0,令和４年 11 月版
1,厚生労働省労働基準局監督課
2,２ はじめに
3,１ 就業規則の意義
4,労働者が安心して働ける明るい職場を作ることは、事業規模や業種を問わず、すべての事業場にとって...
...,...
280,また、 １か月単位の変形労働時間制における週休２日制の形態別の週所定労働時間は、 次の表のと...
281,時間以下となるよう特定の日の労働時間を少なくする等の調整をする必要があります。
282,〔例３〕１年単位の変形労働時間制の規程例 （労働時間及び休憩時間） 第１９条 労働者代表と１...
283,（休日） 第２０条 １年単位の変形労働時間制の適用を受ける労働者の休日については、１年単位の...


## データを永続化させるためにGoogleDriveへ接続
データはノートブックのインスタンスが消えるとなくなってしまうので、学習コストがかかるベクターに変換したデータはドライブへ保存する。

In [12]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


ここでGoogleDriveへのアクセスを許可してください。

In [ ]:
with open("/content/drive/MyDrive/Colab Notebooks/data.txt", "w") as f:
  f.write("Hello drive")

MyDrive/Colab Notebooksフォルダにdata.txtが作成されていることを確認してください。

# 関連規約の抽出

## 就業規則のベクトル化
文字列をベクトル化するOpenAIのget_embedding()関数を使います。これはトークンを消費するので、料金がかかります。

In [ ]:
# このコマンドはTOKENSを消費するので、繰り返し実行しないほうが良い。
# 使用するときはcsvに吐き出したので、pd.read_csv("model-emeddings.csv")を使う
model_df["embedding"] = model_df["１ モデル就業規則"].apply(lambda x: get_embedding(x, engine="text-embedding-ada-002"))
model_df.to_csv("/content/drive/MyDrive/Colab Notebooks/model-embeddings.csv")

NameError: ignored

5分くらいで完了しました。

In [ ]:
ls -lh "/content/drive/MyDrive/Colab Notebooks"

total 11M
-rw------- 1 root root  14K Mar  9 09:58 'chatgpt-api.ipynb のコピー'
-rw------- 1 root root 633K Apr 21 04:29 'COPY_Word Embeddings.ipynb'
-rw------- 1 root root   11 Apr 21 04:21  data.txt
-rw------- 1 root root 9.5M Apr 21 04:32  model-embeddings.csv


続きから始めるときは model-embeddings.csvを読み込むと、早くて料金もかかりません。

学習済みのデータを共有しておきます。

[model-embedding.csv](https://drive.google.com/file/d/1-6U_drUGK7AJ4CHn4Nck5BAb7dQ-yLeL/view?usp=sharing)

## ベクトル化された就業規則データの読み込み

In [ ]:
model_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/model-embeddings.csv").rename({"１ モデル就業規則":"text"}, axis=1)
model_df

,Unnamed: 0,text,embedding
0,0,令和４年 11 月版,"[-0.023388469591736794, -0.020663058385252953,..."
1,1,厚生労働省労働基準局監督課,"[0.0013858058955520391, -0.012126019224524498,..."
2,2,２ はじめに,"[0.009781819768249989, -0.0030580570455640554,..."
3,3,１ 就業規則の意義,"[-0.007960362359881401, -0.010707441717386246,..."
4,4,労働者が安心して働ける明るい職場を作ることは、事業規模や業種を問わず、すべての事業場にとって...,"[0.015495918691158295, -0.017855433747172356, ..."
...,...,...,...
280,280,また、 １か月単位の変形労働時間制における週休２日制の形態別の週所定労働時間は、 次の表のと...,"[-0.002624605083838105, -0.005571787245571613,..."
281,281,時間以下となるよう特定の日の労働時間を少なくする等の調整をする必要があります。,"[-0.016665594652295113, -0.007388188969343901,..."
282,282,〔例３〕１年単位の変形労働時間制の規程例 （労働時間及び休憩時間） 第１９条 労働者代表と１...,"[0.006972527597099543, -0.003516637720167637, ..."
283,283,（休日） 第２０条 １年単位の変形労働時間制の適用を受ける労働者の休日については、１年単位の...,"[0.0009587518288753927, -0.0046020084992051125..."


質問「休暇について」をベクトル化する。次元は1536次元。

In [ ]:
query = "休暇について"
query_vector = get_embedding(query, engine="text-embedding-ada-002")
query_vector

[0.0040262965485453606,
 -0.0011756594758480787,
 0.0027069540228694677,
 -0.02624078281223774,
 -0.008992484770715237,
 0.03391233831644058,
 -0.0218842551112175,
 -0.007779515814036131,
 0.011888369917869568,
 -0.0023433805909007788,
 0.008547941222786903,
 0.00517258420586586,
 0.0012026496697217226,
 -0.022722536697983742,
 -0.001107390271499753,
 -0.02545330487191677,
 0.02464042603969574,
 -0.02626618556678295,
 0.02984793856739998,
 -0.03271842002868652,
 0.00025858951266855,
 0.005829873960465193,
 0.0031276829540729523,
 -0.009087744168937206,
 -0.01859463006258011,
 -0.00303718657232821,
 0.0008978196419775486,
 -0.01440321747213602,
 -0.0019512295257300138,
 -0.009589443914592266,
 0.0484679713845253,
 -0.010230856947600842,
 -0.00412155594676733,
 0.006449060048907995,
 -0.0007469923002645373,
 -0.024094272404909134,
 -0.00833837129175663,
 -0.0060394443571567535,
 0.013018781319260597,
 -0.016321105882525444,
 0.026494808495044708,
 -0.013628441840410233,
 0.00564570585265

In [ ]:
len(_)

1536

In [ ]:
# これはエラーが発生します
model_df["similarities"] = model_df["embedding"].apply(lambda x: cosine_similarity(x, query_vector))
model_df

UFuncTypeError: ignored

エラーが発生する

/usr/local/lib/python3.9/dist-packages/numpy/core/overrides.py in dot(*args, **kwargs)

UFuncTypeError: ufunc 'multiply' did not contain a loop with signature matching types (dtype('<U34412'), dtype('<U23')) -> None

In [ ]:
# エラーが発生しなくなるおまじない
model_df['embedding'] = model_df['embedding'].apply(eval).apply(np.array)

queryとの類似性をベクトルの内積を摂ることにより算出します。

In [ ]:
model_df["similarities"] = model_df["embedding"].apply(lambda x: cosine_similarity(x, query_vector))
model_df.head()

,Unnamed: 0,text,embedding,similarities
0,0,令和４年 11 月版,"[-0.023388469591736794, -0.020663058385252953,...",0.786476
1,1,厚生労働省労働基準局監督課,"[0.0013858058955520391, -0.012126019224524498,...",0.763966
2,2,２ はじめに,"[0.009781819768249989, -0.0030580570455640554,...",0.808099
3,3,１ 就業規則の意義,"[-0.007960362359881401, -0.010707441717386246,...",0.808228
4,4,労働者が安心して働ける明るい職場を作ることは、事業規模や業種を問わず、すべての事業場にとって...,"[0.015495918691158295, -0.017855433747172356, ...",0.819318


関連性順にソートします。

In [ ]:
model_df.sort_values("similarities", ascending=False)

,Unnamed: 0,text,embedding,similarities
75,75,第２４条（年次有給休暇の時間単位での付与）,"[-0.011348715983331203, -0.007791062351316214,...",0.873985
243,243,４ 休日については、毎週少なくとも１回又は４週間を通じ４日以上与えなければなりません（労基法...,"[-0.01608872041106224, -0.001150691881775856, ...",0.872672
8,8,（１） 労働時間関係 始業及び終業の時刻、休憩時間、休日、休暇並びに労働者を２組以上に分けて...,"[-0.0018289131112396717, -0.012719710357487202...",0.872669
96,96,第４３条（休暇等の賃金）,"[0.005117868073284626, -0.003966835327446461, ...",0.869345
240,240,第４章 労働時間、休憩及び休日 １ 労働時間、休憩及び休日に関することは、就業規則の絶対的必...,"[-0.0047806790098547935, -0.000381161138648167...",0.864110
...,...,...,...,...
135,135,第１３章 公益通報者保護…………………………………………８９,"[-0.01092766597867012, -0.02020331658422947, 0...",0.747587
46,46,第３章 服務規律……………………………………………………１６,"[0.006817985791712999, -0.009270594455301762, ...",0.747393
123,123,９,"[0.008061571046710014, -0.005975430831313133, ...",0.747032
144,144,）は、労働基準法（以下「労基法」とい,"[-0.014768361113965511, -0.012791063636541367,...",0.744001


関連順にソートできました。この中の条項の部分だけを抜き出すと、就業規則チャットボットになります。 関連順トップ3をつなげて表示します。

In [ ]:
top3 = model_df.sort_values("similarities", ascending=False).text[:3].values; top3

array(['第２４条（年次有給休暇の時間単位での付与）',
       '４ 休日については、毎週少なくとも１回又は４週間を通じ４日以上与えなければなりません（労基法第３５条）。',
       '（１） 労働時間関係 始業及び終業の時刻、休憩時間、休日、休暇並びに労働者を２組以上に分けて交替に就業させる場合においては就業時転換に関する事項'],
      dtype=object)

In [ ]:
" ".join(top3)

'第２４条（年次有給休暇の時間単位での付与） ４ 休日については、毎週少なくとも１回又は４週間を通じ４日以上与えなければなりません（労基法第３５条）。 （１） 労働時間関係 始業及び終業の時刻、休憩時間、休日、休暇並びに労働者を２組以上に分けて交替に就業させる場合においては就業時転換に関する事項'

トップ３と言わず、similarities0.8以上などのしきい値を設けて、
トークン数が許す限りの情報を列挙してChatGPTに渡して、日本語として整形してもらうと完成度の高いChatBOTになりそうな気がします。

## 関連性順の上位で規約を絞り込む
就業規則に関する質問を受け付けて関連する回答トップ３の文字列を結合して返す関数を示します。

In [ ]:
def regulation_search(vector_df):
  """就業規則に関する質問を与えると、関連性の高い条項を検索して返す"""
  query = input("就業規則に関して何でも聞いてください。: ")
  query_vector = get_embedding(query, engine="text-embedding-ada-002")
  vector_df["similarities"] = vector_df["embedding"].apply(lambda x: cosine_similarity(x, query_vector))
  top3 = vector_df.sort_values("similarities", ascending=False).text[:3].values
  return " ".join(top3)


In [ ]:
regulation_search(model_df)

就業規則に関して何でも聞いてください。: 年間休日は何日ですか


'（休日） 第２０条 休日は、次のとおりとする。① 日曜日 ② 年 月 日を起算日とする２週間ごとの第２ 土曜日 ③ 国民の祝日（日曜日と重なったときは翌日） ④ 年末年始（１２月 日～１月 日） ⑤ 夏季休日（ 月 日～ 月 日） ⑥ その他会社が指定する日 ２ 休日は、原則として暦日（午前０時から午後１２時までの継続２４時間をいう。）で与えなければなりません。しかし、番方編成による交替制（８時間３交替勤務のような場合をいう。）を導入するような場合、以下の要件を満たせば休日は暦日ではなく、継続した２４時間を与えれば差し支えないとされています（昭和６３年３月１４日付け基発１５０号）。 第２０条 休日は、次のとおりとする。 ① 土曜日及び日曜日 ② 国民の祝日（日曜日と重なったときは翌日） ③ 年末年始（１２月 日～１月 日） ④ 夏季休日（ 月 日～ 月 日） ⑤ その他会社が指定する日'

In [ ]:
regulation_search(model_df)

就業規則に関して何でも聞いてください。: 昇給はありますか


'第４９条（昇給） １ 労働者を雇い入れるに際し、労働者に賃金、労働時間、その他の労働条件を明示することが必要です。特に、労働条件を明示するに当たり、次の（１）から（６）までの項目（昇給に関する事項を除く）については、原則書面の交付により明示する必要があります（労基法第１５条、労基則第５条）。 （１） 労働契約の期間に関する事項 （２） 期間の定めのある労働契約を更新する場合の基準に関する事項（期間の定めのある労働契約を更新する場合に限る） （３） 就業の場所及び従事すべき業務に関する事項 （４） 始業及び終業の時刻、所定労働時間を超える労働の有無、休憩時間、休日、休 暇並びに交替制により就業させる場合における就業時転換に関する事項 （５） 賃金（退職手当及び臨時に支払われる賃金等を除く。）の決定、計算及び支払の方法、賃金の締切り及び支払の時期並びに昇給に関する事項 である。'

In [ ]:
regulation_search(model_df)

就業規則に関して何でも聞いてください。: 一日何時間労働ですか


'２ １日の所定労働時間は、７ 時間１５分とする。 （労働時間及び休憩時間） 第１９条 １週間の所定労働時間は、 年 月 日を起算日として、２週間ごとに平均して、１週間当たり４０時間とする。 ２ 本規程例は、１日の所定労働時間を固定していますが、業務の都合等によって日々の所定労働時間を変えることもできます。この場合も、一定期間を平均して１週当たりの労働時間が４０時間を超えないようにしなければなりません。'

回答はmilisecオーダーで返ってきて軽快です。関連条項を抽出する前は文字列のベクトル化の比較を行っているだけなので、OpenAI APIを使っていません。つまり、一度モデルを作ってしまえば(ここまで有料)、回答の抽出は無料です。

In [ ]:
regulation_search(model_df)

就業規則に関して何でも聞いてください。: 退職金の計算方法


'（１） 退職手当関係 適用される労働者の範囲、退職手当の決定、計算及び支払の方法並びに退職手当の支払の時期に関する事項 第５６条（退職金の支払方法及び支払時期） 第５５条（退職金の額）'

## 関連性で規約を絞りこむ
関連性0.85以上の規約のみを抜き出します。

In [ ]:
def regulation_search(vector_df, query:str, threshold=0.85, verbose=False) -> list[str]:
  """就業規則に関する質問を与えると、関連性の高い条項を検索して返す
  verbose=Trueでテキストと類似度を表示する。
  """
  query_vector = get_embedding(query, engine="text-embedding-ada-002")
  vector_df["similarities"] = vector_df["embedding"].apply(lambda x: cosine_similarity(x, query_vector))
  df = vector_df.sort_values("similarities", ascending=False)  # 関連性順にソート
  df = df[df.similarities >= threshold]  # 関連性の高いものだけ抽出
  if verbose:
    print(df.loc[:,["text", "similarities"]])
  return df.text.values

# regulation_search()をDataFrameのメソッドとして扱う
setattr(pd.DataFrame, "regulation_search", regulation_search)

In [ ]:
model_df.regulation_search("休暇は何曜日？", verbose=True)

                                                  text  similarities
257  第２０条 休日は、次のとおりとする。 ① 土曜日及び日曜日 ② 国民の祝日（日曜日と重なった...      0.882269
271  （休日） 第２０条 休日は、次のとおりとする。① 日曜日 ② 年 月 日を起算日とする２週間...      0.878646
259  【第２０条 休日】 １ 労基法では何曜日を休日とするかあるいは国民の祝日を休日とするかについ...      0.870682
266  ② 休日は労働者の労働義務のない日ですから、これを振り替える場合は、以下に示す措置が必要とな...      0.857906
243  ４ 休日については、毎週少なくとも１回又は４週間を通じ４日以上与えなければなりません（労基法...      0.855543
284  ２ １年単位の変形労働時間制を適用しない労働者の休日については、以下のとおり指定し、月間休日...      0.854526


array(['第２０条 休日は、次のとおりとする。 ① 土曜日及び日曜日 ② 国民の祝日（日曜日と重なったときは翌日） ③ 年末年始（１２月 日～１月 日） ④ 夏季休日（ 月 日～ 月 日） ⑤ その他会社が指定する日',
       '（休日） 第２０条 休日は、次のとおりとする。① 日曜日 ② 年 月 日を起算日とする２週間ごとの第２ 土曜日 ③ 国民の祝日（日曜日と重なったときは翌日） ④ 年末年始（１２月 日～１月 日） ⑤ 夏季休日（ 月 日～ 月 日） ⑥ その他会社が指定する日',
       '【第２０条 休日】 １ 労基法では何曜日を休日とするかあるいは国民の祝日を休日とするかについて規定していません。１週間の中で何曜日を休日としても、また、週によって異なる曜日を休日',
       '② 休日は労働者の労働義務のない日ですから、これを振り替える場合は、以下に示す措置が必要となります。 ア 就業規則に振替休日の規程を置くこと。 イ 振替休日は特定すること。 ウ 振替休日は４週４日の休日が確保される範囲のできるだけ近接した日とすること。 エ 振替は前日までに通知すること。',
       '４ 休日については、毎週少なくとも１回又は４週間を通じ４日以上与えなければなりません（労基法第３５条）。',
       '２ １年単位の変形労働時間制を適用しない労働者の休日については、以下のとおり指定し、月間休日カレンダーに定め、対象期間の初日の３０日前までに各労働者に通知する。 ① 日曜日（前条第３号の特定期間を除く。） ② 国民の祝日（日曜日と重なったときは翌日） ③ 年末年始（１２月 日～１月 日） ④ 夏季休日（ 月 日～ 月 日） ⑤ その他会社が指定する日'],
      dtype=object)

In [ ]:
model_df.regulation_search("昇給について")

array(['第４９条（昇給）',
       '１ 労働者を雇い入れるに際し、労働者に賃金、労働時間、その他の労働条件を明示することが必要です。特に、労働条件を明示するに当たり、次の（１）から（６）までの項目（昇給に関する事項を除く）については、原則書面の交付により明示する必要があります（労基法第１５条、労基則第５条）。 （１） 労働契約の期間に関する事項 （２） 期間の定めのある労働契約を更新する場合の基準に関する事項（期間の定めのある労働契約を更新する場合に限る） （３） 就業の場所及び従事すべき業務に関する事項 （４） 始業及び終業の時刻、所定労働時間を超える労働の有無、休憩時間、休日、休 暇並びに交替制により就業させる場合における就業時転換に関する事項 （５） 賃金（退職手当及び臨時に支払われる賃金等を除く。）の決定、計算及び支払の方法、賃金の締切り及び支払の時期並びに昇給に関する事項',
       '（２） 賃金関係 賃金の決定、計算及び支払の方法、賃金の締切り及び支払の時期並びに昇給に関する事項'], dtype=object)

In [ ]:
model_df.regulation_search("休暇はいつですか")

array(['第２０条 休日は、次のとおりとする。 ① 土曜日及び日曜日 ② 国民の祝日（日曜日と重なったときは翌日） ③ 年末年始（１２月 日～１月 日） ④ 夏季休日（ 月 日～ 月 日） ⑤ その他会社が指定する日',
       '４ 休日については、毎週少なくとも１回又は４週間を通じ４日以上与えなければなりません（労基法第３５条）。',
       '（休日） 第２０条 休日は、次のとおりとする。① 日曜日 ② 年 月 日を起算日とする２週間ごとの第２ 土曜日 ③ 国民の祝日（日曜日と重なったときは翌日） ④ 年末年始（１２月 日～１月 日） ⑤ 夏季休日（ 月 日～ 月 日） ⑥ その他会社が指定する日',
       '第２４条（年次有給休暇の時間単位での付与）',
       '② 休日は労働者の労働義務のない日ですから、これを振り替える場合は、以下に示す措置が必要となります。 ア 就業規則に振替休日の規程を置くこと。 イ 振替休日は特定すること。 ウ 振替休日は４週４日の休日が確保される範囲のできるだけ近接した日とすること。 エ 振替は前日までに通知すること。'],
      dtype=object)

## トークンサイズで規約を絞りこむ
関連性が高いだけのものだけに絞っても、ChatGPTへ渡すにはデータサイズが大きすぎる場合があります。ChatGPTモデル`gpt-3.5-turbo`と会話するためには、回答のトークン(`max_tokens`)と質問のトークン数の合計が4096に収まるようにしなければHTTP400エラーとなります。

[OpenAI Community Forum - Error retrieving ‘completions’: 400 Bad Request](https://community.openai.com/t/error-retrieving-completions-400-bad-request/34004)

> A common reason of the error 400 is that the prompt exceed the limit of character allowed by the API.
>
> I recommend you check the API documentation to see if there are any limits on the size of the prompt or the number of items in the question.

これ以降では特に断りがない限りモデルを`gpt-3.5-turbo`とします。

各textのトークンサイズを取得するには、`tiktoken`モジュールの`encode`メソッドを各行に`apply`します。

In [ ]:
enc = tiktoken.encoding_for_model("gpt-3.5-turbo")
model_df["token_length"] = model_df.text.apply(lambda x: len(enc.encode(x)))
model_df

,Unnamed: 0,text,embedding,similarities,token_length
0,0,令和４年 11 月版,"[-0.023388469591736794, -0.020663058385252953,...",0.771083,10
1,1,厚生労働省労働基準局監督課,"[0.0013858058955520391, -0.012126019224524498,...",0.786791,22
2,2,２ はじめに,"[0.009781819768249989, -0.0030580570455640554,...",0.766051,6
3,3,１ 就業規則の意義,"[-0.007960362359881401, -0.010707441717386246,...",0.810579,13
4,4,労働者が安心して働ける明るい職場を作ることは、事業規模や業種を問わず、すべての事業場にとって...,"[0.015495918691158295, -0.017855433747172356, ...",0.813434,164
...,...,...,...,...,...
280,280,また、 １か月単位の変形労働時間制における週休２日制の形態別の週所定労働時間は、 次の表のと...,"[-0.002624605083838105, -0.005571787245571613,...",0.819257,668
281,281,時間以下となるよう特定の日の労働時間を少なくする等の調整をする必要があります。,"[-0.016665594652295113, -0.007388188969343901,...",0.826428,35
282,282,〔例３〕１年単位の変形労働時間制の規程例 （労働時間及び休憩時間） 第１９条 労働者代表と１...,"[0.006972527597099543, -0.003516637720167637, ...",0.821624,524
283,283,（休日） 第２０条 １年単位の変形労働時間制の適用を受ける労働者の休日については、１年単位の...,"[0.0009587518288753927, -0.0046020084992051125...",0.838837,170


In [ ]:
def regulation_search(vector_df, query:str, threshold=0.85, model="gpt-3.5-turbo", verbose=False) -> list[str]:
  """就業規則に関する質問を与えると、関連性の高い条項を検索して返す
  verbose=Trueでテキストと類似度を表示する。
  """
  # 質問のベクトル化
  query_vector = get_embedding(query, engine="text-embedding-ada-002")
  # 各テキストのトークンの長さを取得
  enc = tiktoken.encoding_for_model(model)
  vector_df["token_length"] = vector_df.text.apply(lambda x: len(enc.encode(x)))
  # 関連性を算出
  vector_df["similarities"] = vector_df["embedding"].apply(lambda x: cosine_similarity(x, query_vector))
  df = vector_df.sort_values("similarities", ascending=False)  # 関連性順にソート
  return df[df.similarities >= threshold]  # 関連性の高いものだけ抽出

# regulation_search()をDataFrameのメソッドとして扱う
setattr(pd.DataFrame, "regulation_search", regulation_search)

In [ ]:
model_df.regulation_search("年間休日は何日ですか？")

,Unnamed: 0,text,embedding,similarities,token_length
271,271,（休日） 第２０条 休日は、次のとおりとする。① 日曜日 ② 年 月 日を起算日とする２週間...,"[0.00294488575309515, 0.0012329686433076859, 0...",0.869244,135
261,261,２ 休日は、原則として暦日（午前０時から午後１２時までの継続２４時間をいう。）で与えなければ...,"[-0.010599075816571712, -0.008891886100172997,...",0.861892,168
268,268,（労働時間及び休憩時間） 第１９条 １週間の所定労働時間は、 年 月 日を起算日として、２週...,"[-0.001058968249708414, -0.005311335902661085,...",0.861786,72
257,257,第２０条 休日は、次のとおりとする。 ① 土曜日及び日曜日 ② 国民の祝日（日曜日と重なった...,"[-0.0018275119364261627, 0.0001700313005130738...",0.861266,111
243,243,４ 休日については、毎週少なくとも１回又は４週間を通じ４日以上与えなければなりません（労基法...,"[-0.01608872041106224, -0.001150691881775856, ...",0.861178,56
259,259,【第２０条 休日】 １ 労基法では何曜日を休日とするかあるいは国民の祝日を休日とするかについ...,"[0.010833602398633957, 0.0028621950186789036, ...",0.859966,94
283,283,（休日） 第２０条 １年単位の変形労働時間制の適用を受ける労働者の休日については、１年単位の...,"[0.0009587518288753927, -0.0046020084992051125...",0.856489,170
75,75,第２４条（年次有給休暇の時間単位での付与）,"[-0.011348715983331203, -0.007791062351316214,...",0.852766,24
284,284,２ １年単位の変形労働時間制を適用しない労働者の休日については、以下のとおり指定し、月間休日...,"[-0.000653393508400768, -0.003440316766500473,...",0.851563,192


`regulation_search`関数を改造して、textに対して関連性とトークン数を表示したデータフレームを得られました。

# ChatGPT連携
先の項目で類似度(similarities)0.85以上の項目だけ抽出し、さらにトークン数を算出しました。
これを制限トークン内に収めるようにテキストを結合したものをChatGPTに投げ、ChatGPTにいい感じの日本語に直してもらえばチャットボットです。

## トークン数の制限 

システムロール(role:systemのcontent)、質問、回答(max_token)の合計トークン数が4096以内に収まっていなければなりません。

次のようなsystem_roleを与えるとします。トークンサイズは94です。

In [ ]:
system_role = f"""あなたは就業規則に関するエキスパートです。
以下に与える`関連規約`を参照し、質問に対して平易な日本語で答えてください。
就業規則以外のことには答えられません。

# 関連規約
"""
len(enc.encode(system_role))

94

In [ ]:
query = "年間休日は何日ですか？"
related_rules = model_df.regulation_search(query)
related_rules[related_rules.token_length.cumsum() < 500]


,Unnamed: 0,text,embedding,similarities,token_length
271,271,（休日） 第２０条 休日は、次のとおりとする。① 日曜日 ② 年 月 日を起算日とする２週間...,"[0.00294488575309515, 0.0012329686433076859, 0...",0.869374,135
261,261,２ 休日は、原則として暦日（午前０時から午後１２時までの継続２４時間をいう。）で与えなければ...,"[-0.010599075816571712, -0.008891886100172997,...",0.861999,168
268,268,（労働時間及び休憩時間） 第１９条 １週間の所定労働時間は、 年 月 日を起算日として、２週...,"[-0.001058968249708414, -0.005311335902661085,...",0.861885,72
257,257,第２０条 休日は、次のとおりとする。 ① 土曜日及び日曜日 ② 国民の祝日（日曜日と重なった...,"[-0.0018275119364261627, 0.0001700313005130738...",0.861406,111


In [ ]:
sum(_.token_length)

486

仮に合計トークン数が500以内になるように出力させました。

In [ ]:
query = "年間休日は何日？"
system_role = f"""あなたは就業規則に関するエキスパートです。
以下に与える`関連規約`を参照し、質問に対して平易な日本語で答えてください。
就業規則以外のことには答えられません。

# 関連規約
"""

# 質問とシステムロールのトークン長さ
query_length = len(enc.encode(query))
system_length = len(enc.encode(system_role))
print("query_length, system_length: ", query_length, system_length)  # 9, 94

# 関連性の高い規約のみ抽出
related_rules = model_df.regulation_search(query)
print("cumsum rules tokens:",related_rules.token_length.sum())  # 2076

# ChatGPT 3.5 turboに送るトークン数が制限内に収まるように調整
max_tokens = 1000
token_limit = 4096 - max_tokens - query_length - system_length
print("token_limit:",token_limit)  # 2993
related_rules[related_rules.token_length.cumsum() <= token_limit]


query_length, system_length:  9 94
cumsum rules tokens: 2076
token_limit: 2993


,Unnamed: 0,text,embedding,similarities,token_length
271,271,（休日） 第２０条 休日は、次のとおりとする。① 日曜日 ② 年 月 日を起算日とする２週間...,"[0.00294488575309515, 0.0012329686433076859, 0...",0.880660,135
257,257,第２０条 休日は、次のとおりとする。 ① 土曜日及び日曜日 ② 国民の祝日（日曜日と重なった...,"[-0.0018275119364261627, 0.0001700313005130738...",0.873072,111
261,261,２ 休日は、原則として暦日（午前０時から午後１２時までの継続２４時間をいう。）で与えなければ...,"[-0.010599075816571712, -0.008891886100172997,...",0.872378,168
259,259,【第２０条 休日】 １ 労基法では何曜日を休日とするかあるいは国民の祝日を休日とするかについ...,"[0.010833602398633957, 0.0028621950186789036, ...",0.871654,94
243,243,４ 休日については、毎週少なくとも１回又は４週間を通じ４日以上与えなければなりません（労基法...,"[-0.01608872041106224, -0.001150691881775856, ...",0.871527,56
268,268,（労働時間及び休憩時間） 第１９条 １週間の所定労働時間は、 年 月 日を起算日として、２週...,"[-0.001058968249708414, -0.005311335902661085,...",0.871388,72
283,283,（休日） 第２０条 １年単位の変形労働時間制の適用を受ける労働者の休日については、１年単位の...,"[0.0009587518288753927, -0.0046020084992051125...",0.870246,170
74,74,第２３条（年次有給休暇）,"[-0.015378488227725029, -0.00465159909799695, ...",0.864643,15
75,75,第２４条（年次有給休暇の時間単位での付与）,"[-0.011348715983331203, -0.007791062351316214,...",0.864516,24
284,284,２ １年単位の変形労働時間制を適用しない労働者の休日については、以下のとおり指定し、月間休日...,"[-0.000653393508400768, -0.003440316766500473,...",0.864279,192


トークン制限が2993に対して、しきい値以上の関連規約をすべて合計しても2076しかないようなので、この質問では`cumsum()`によるフィルターが機能してません。
これを関数化して、いくつか質問をぶつけてみましょう。

In [ ]:
def filter_by_token_length(vector_df:pd.DataFrame,
                           query:str,
                           system_role:str,
                           max_tokens=1000,
                           model="gpt-3.5-turbo", 
                           model_token_limit=4096,
                           verbose=False,
                           *args, **kwargs):
  """質問とシステムロールプロンプトを加味して、トークン数を制限して規約の入ったデータフレームを返す"""
  # 質問とシステムロールのトークン長さ
  enc = tiktoken.encoding_for_model(model)
  query_length = len(enc.encode(query))
  system_length = len(enc.encode(system_role))
  
  # 関連性の高い規約のみ抽出
  related_rules = model_df.regulation_search(query, *args, **kwargs)
  
  # ChatGPT 3.5 turboに送るトークン数が制限内に収まるように調整
  token_limit = model_token_limit - max_tokens - query_length - system_length
  if verbose:
    print("query_length, system_length: ", query_length, system_length)
    print("cumsum rules tokens:",related_rules.token_length.sum())
    print("token_limit:", token_limit)
  return related_rules[related_rules.token_length.cumsum() <= token_limit]
  
# DataFrameのメソッドとして扱う
setattr(pd.DataFrame, "filter_by_token_length", filter_by_token_length)

In [ ]:
system_role = f"""あなたは就業規則に関するエキスパートです。
以下に与える`関連規約`を参照し、質問に対して平易な日本語で答えてください。
就業規則以外のことには答えられません。
最後に必ず関連する規約の番号を案内してください。

# 関連規約
"""

model_df.filter_by_token_length("産休制度はありますか", system_role, verbose=True)

query_length, system_length:  10 116
cumsum rules tokens: 221
token_limit: 2970


,Unnamed: 0,text,embedding,similarities,token_length
76,76,第２５条（産前産後の休業）,"[-0.017860770225524902, -0.007786657195538282,...",0.867094,17
57,57,[例１] 完全週休２日制を採用する場合の規程例,"[-0.009088310413062572, 0.00024845494772307575...",0.854824,26
265,265,その一方で、代休は、定められた法定休日に休日労働を行わせた場合ですから、その後に代休を与えて...,"[-0.007885788567364216, -0.012739572674036026,...",0.852569,109
75,75,第２４条（年次有給休暇の時間単位での付与）,"[-0.011348715983331203, -0.007791062351316214,...",0.852131,24
193,193,３ 休職の定義、休職期間の制限、復職等については、労基法に定めはありません。,"[-0.016382284462451935, -0.020896287634968758,...",0.850097,45


In [ ]:
model_df.filter_by_token_length("定年は何歳？", system_role,  verbose=True)

query_length, system_length:  6 94
cumsum rules tokens: 25
token_limit: 2996


,Unnamed: 0,text,embedding,similarities,token_length
106,106,[例１] 定年を満６５歳とする例,"[0.0002256531297462061, -0.011469286866486073,...",0.869023,16
105,105,第５１条（定年等）,"[-0.009599292650818825, -0.015321693383157253,...",0.861034,9


関連規約を多く出力するためにしきい値を0.6まで下げます。

In [ ]:
model_df.filter_by_token_length("定年は何歳？", system_role, verbose=True, threshold=0.6)


query_length, system_length:  6 94
cumsum rules tokens: 23176
token_limit: 2996


,Unnamed: 0,text,embedding,similarities,token_length
106,106,[例１] 定年を満６５歳とする例,"[0.0002256531297462061, -0.011469286866486073,...",0.869023,16
105,105,第５１条（定年等）,"[-0.009599292650818825, -0.015321693383157253,...",0.861034,9
110,110,［例４］ 定年を満６５歳とし、その後希望者の意向を踏まえて継続雇用または業務,"[-0.012937641702592373, -0.04587045684456825, ...",0.849548,49
107,107,[例２] 定年を満６０歳とし、その後希望者を再雇用する例,"[-0.016348684206604958, -0.023294826969504356,...",0.846568,30
174,174,４ 試用期間は、勤続年数に通算する。,"[-0.00849276315420866, -0.014469152316451073, ...",0.834495,20
...,...,...,...,...,...
100,100,第４７条（賃金の支払と控除）,"[-0.008588608354330063, 0.0070113795809447765,...",0.779203,17
137,137,第１４章 副業・兼業………………………………………………９０,"[0.0007647178135812283, -0.027631524950265884,...",0.779043,19
124,124,第６０条（長時間労働者に対する面接指導）,"[-0.02042042650282383, 0.005226736888289452, 0...",0.778799,23
210,210,【第１６条 個人情報保護】 個人情報の保護に関する法律（平成１５年法律第５７号）の全面施行に...,"[-0.010957937687635422, 0.008579668588936329, ...",0.778618,88


In [ ]:
_.token_length.sum()

2987

フィルター前のtoken_lengthが2万超え、limitが2996に対して、出てきたデータフレームのtoken_lengthの合計は2987なので、フィルターが効いていますね。

## 関連規則をChatGPTに参照させる
質問に関連する規約だけをトークン制限の許す限り抽出し、ChatGPTに渡してわかりやすく答えてもらいます。

In [ ]:
system_role = f"""あなたは就業規則に関するエキスパートです。
以下に与える`関連規約`を参照し、質問に対して平易な日本語で答えてください。
就業規則以外のことには答えられません。
最後に必ず関連する規約の番号を案内してください。

# 関連規約
"""
query = "産休制度はありますか"

# 関連規約の抽出
filtered_content = "\n".join(model_df.filter_by_token_length(query, system_role).text)

# ChatGPTに規約を渡していい感じに日本語に直してもらう
messages = [
    {
        "role": "system",
        "content": system_role + filtered_content  # システムプロンプトに関連規約を追加
    },
    {
        "role": "user",
        "content": query
    }
]
completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    max_tokens=1000,
    temperature=0.2,
    messages=messages,
)
content = completion.choices[0].message.content
content

'はい、労働基準法により、女性従業員に対して産前産後の休業を取得する権利があります。具体的には、出産予定日の６週間前から産後８週間までの期間について、休業を取得することができます。また、休業期間中には、給与の一部が支給されることになっています。詳細については、労働基準法第２５条を参照してください。'

In [ ]:
def ask_regulation(vector_df:pd.DataFrame, query:str):
  """usage:
    model_df.ask_regulation("質問")
  """
  system_role = f"""あなたは就業規則に関するエキスパートです。
  以下に与える`関連規約`を参照し、質問に対して平易な日本語で答えてください。
  就業規則以外のことには答えられません。
  最後に必ず関連する規約の番号を案内してください。
  
  # 関連規約
  """
  # 関連規約の抽出
  regulations = vector_df.filter_by_token_length(query, system_role).text
  filtered_content = "\n".join(regulations)
  
  # ChatGPTに規約を渡していい感じに日本語に直してもらう
  messages = [
      {
          "role": "system",
          "content": system_role + filtered_content  # システムプロンプトに関連規約を追加
      },
      {
          "role": "user",
          "content": query
      }
  ]
  completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      max_tokens=1000,
      temperature=0.2,
      messages=messages,
  )
  content = completion.choices[0].message.content
  print(len(enc.encode(content)), "tokens")
  return content

# DataFrameのメソッドとして扱う
setattr(pd.DataFrame, "ask_regulation", ask_regulation)

In [ ]:
model_df.ask_regulation("何時間働けば良い？")

255 tokens


'質問からは具体的な条件が分からないため、一般的な回答として、1日の所定労働時間は7時間15分であり、1週間の所定労働時間は40時間となっています。また、1日の労働時間が6時間を超える場合には少なくとも45分、8時間を超える場合には少なくとも1時間の休憩時間を与える必要があります。これらの規定に従って、労働時間と休憩時間を調整し、1週間あたり40時間以内に収めるように働く必要があります。\n\n関連規約：労働時間及び休憩時間に関する規定（労働時間及び休憩時間）第19条、休憩時間に関する規定（労基法第34条）'

In [ ]:
model_df.ask_regulation("産休制度はある？")

189 tokens


'就業規則によっては、産前産後の休業制度が定められている場合があります。産前産後の休業は、女性従業員が出産前後の一定期間休業することができる制度であり、産前休業と産後休業に分かれています。ただし、具体的な規定は就業規則によって異なるため、詳細については就業規則を確認する必要があります。\n\n関連規約：第２５条（産前産後の休業）'

In [ ]:
model_df.ask_regulation("振替休日とはどのような場合に発生しますか")


160 tokens


'振替休日とは、例えば業務の都合によって所定休日である日曜日に勤務させなければならない場合に、当該日曜日を勤務日に変更し、その代わり勤務日である例えば月曜日を休日とするように、所定の休日とあらかじめ他の勤務日と振り替えることをいいます。つまり、法定休日を他の日に振り替えることです。'

In [ ]:
model_df.ask_regulation("始業、終業の時刻は誰がどのように管理しますか")

286 tokens


'労働者の始業・終業時刻は、使用者が確認し、記録することが原則です。具体的には、使用者が労働者の始業・終業時刻を確認し、これを記録することが求められます。記録方法については、原則としてタイムカードや勤怠管理システムなどを使用することが推奨されています。ただし、自己申告制により始業・終業時刻の確認及び記録を行う場合には、別途措置が必要とされています（関連規約：４、３、５、第１７条（始業及び終業時刻の記録）、（ロ）２番（準夜勤）始業・終業時刻 休憩時間始業 午前 時 分時 分から 時 分まで終業 午後 時 分）。'

In [ ]:
model_df.ask_regulation("明日の天気はなんですか")

87 tokens


'申し訳ありませんが、私は就業規則に関するエキスパートであり、天気予報に関する情報を持っていません。お手数ですが、天気予報サイトやアプリをご確認ください。関連する規約はありません。'

## 関連規約をChatGPTに参照させて答えさせる流れをクラス化
ここまでの流れを一旦整理します。
- 就業規則データを読み込んで、ベクトル化する。
- ベクトル化には時間とお金がかかるので、完了したらストレージに保存しておく。
- 再開するときはデータを読み込むところから。
- 質問の内容から、関連度合いを調べてある程度関連しそうな規約だけを抽出する。
- システムプロンプトに関連規約を埋め込む。
- システムプロンプトと質問と回答の合計トークン数が4096以下になるように調整してChatGPTへ渡す。

合計トークンの内訳は以下のようになります。
- 回答の最大トークンは`max_tokens`というプロパティで決められている。どういうアシスタントを想定しているかにもよるが、300-1000トークンくらいが妥当か。
- システムプロンプトの規約以外の部分は予め決まっている。約100トークン。`あなたは就業規則のエキスパートです。...`
- 関連規約のトークン数はアシスタントに知識をどれだけ与えたいかによるが、1500-2500トークン程度が妥当か。
- これまでのトークン数を4096から引いたトークン数がユーザーに与えられる質問および前のやり取りに使えるトークン数となる。400-2200トークン程度。

以上から、トークン数の内訳を次のように設計し直します。
* アシスタントの回答`max_tokens=500`
* システムプロンプトの規約以外に116トークン
* システムプロンプトの規約部分に2000トークン
* 残りの1480トークンをユーザーの質問および以前のやり取りに使用。
    * やりとりが1480トークンを超えたら過去の会話から削除
    * ユーザーの質問だけで1480トークンを超えたら`質問が長すぎます`というエラーを発生させる。

### 便利な実装
実装の前に、実装が便利になるクラスを実装していきます。

#### Roleクラスの実装

In [5]:
from enum import Enum
class Role(str,Enum):
    System = "system"
    Assistant = "assistant"
    User = "user"

    def __str__(self):
        return self.name.lower()

In [6]:
Role.System == "system"

True

In [7]:
Role.System

<Role.System: 'system'>

In [8]:
# 使い方
str(Role.System)

'system'

`namedtuple`で実装する場合、strを入れないと小文字のstrにならない

```python
from collections import namedtuple
Message = namedtuple("Message", ["role", "content"])
# strを入れないと小文字のstrにならない
assert Message(str(Role.System), "あなたは就業規則に関する") == ('system','あなたは就業規則に関する')
```

#### Messageクラスの実装

In [9]:
from dataclasses import dataclass
@dataclass
class Message:
    role:Role
    content:str

    def _asdict(self):
        return {"role":str(self.role), "content": self.content}


In [10]:
# 使い方
messages = [
    Message(Role.System, "あなたは就業規則に関する"),
    Message(Role.User, "質問")
]
[i._asdict() for i in messages]

[{'role': 'system', 'content': 'あなたは就業規則に関する'},
 {'role': 'user', 'content': '質問'}]

## 関連規約の参照メソッド、規約に基づいた回答の考案メソッドを持ったクラス
`pd.DataFrame`を継承した`VectorFrame`クラスを実装しました。ChatGPTのモデル、APIのプロパティ、各トークン数のプロパティを追加しました。

In [25]:
class VectorFrame(pd.DataFrame):
    system_prompt = """あなたは就業規則に関するエキスパートです。
    以下に与える`関連規約`を参照し、質問に対して平易な日本語で答えてください。
    就業規則以外のことには答えられません。
    最後に必ず関連する規約の番号を案内してください。

    # 関連規約
    """  # 116 tokens

    def __init__(
            self,
            *args,
            model="gpt-3.5-turbo",
            model_token_limit=4096,
            max_tokens=500,
            temperature=0.2,
            system_max_tokens=2000,  # system promptは合計で2116
            user_max_tokens=1480,
            **kwargs):
        super().__init__(*args, **kwargs)
        self.model = model
        self.model_token_limit = model_token_limit
        self.max_tokens = max_tokens
        self.temperature = temperature
        self.system_max_tokens = system_max_tokens
        self.user_max_tokens = user_max_tokens
        # 各テキストのトークンの長さを取得
        self.enc = tiktoken.encoding_for_model(self.model)
        self["token_length"] = self.text.apply(
            lambda x: len(self.enc.encode(x)))

    def get_token_length(self, *args: str):
        s = '\n'.join(args)
        return len(self.enc.encode(s))

    def search_regulation(self, query: str, threshold=0.85, verbose=False):
        """就業規則に関する質問を与えると、関連性の高い規約を検索して返す
        トークン数がデフォルトで2000未満になるようにして返す。
        verbose=Trueでテキストと類似度を表示する。
        """
        # 質問のベクトル化
        query_vector = get_embedding(query, engine="text-embedding-ada-002")
        # 関連性を算出
        self["similarities"] = self["embedding"].apply(
            lambda x: cosine_similarity(x, query_vector))
        # 関連性順にソート
        self.sort_values("similarities", ascending=False, inplace=True)
        # 関連性の高いものだけ抽出 かつ 2000トークン以内に収まるようにフィルター
        filtered = self[(self.similarities >= threshold) &
                        (self.token_length.cumsum() <= self.system_max_tokens)]
        if verbose:
            print("related rules:", filtered)
            print("cumsum rules tokens:", filtered.token_length.sum())
        return VectorFrame(filtered)

    def ask_regulation(self, query: str, messages:list[Message]=[],
                       *args, **kwargs) -> str:
        """usage:
          model_df.ask_regulation("質問")
        """
        user_prompt_length = self.get_token_length(query)
        assert user_prompt_length <= self.user_max_tokens,\
            "システムプロンプトは1480トークン以下: {user_prompt_length}"
        # システムプロンプトに関連規約を追加
        # list()をつけないと各行にsystem_promptが追加されてしまう
        system_prompt = "\n".join([VectorFrame.system_prompt] +
                                  list(self.text))
        system_prompt_length = self.get_token_length(system_prompt)
        assert system_prompt_length <= 2116,\
            f"システムプロンプトは2116トークン以下 {system_prompt_length}"

        # システムプロンプトと質問をプロンプトに追加
        messages.insert(0, Message(Role.System, system_prompt))
        messages.append(Message(Role.User, query))
        # プロンプトのトークン数が制限に収まるように調整
        while True:
            all_prompt_length = self.get_token_length(*(i.content for i in messages))
            if all_prompt_length <= self.model_token_limit:
                break
            messages.pop(1)  # System promptを除いて上から順に会話履歴を削除
        assert all_prompt_length <= self.model_token_limit,\
            f"全プロンプトの合計は4096トークン以下 {all_prompt_length}"

        # ChatGPTに規約を渡していい感じに日本語に直してもらう
        completion = openai.ChatCompletion.create(
            model=self.model,
            max_tokens=self.max_tokens,
            temperature=self.temperature,
            messages=[i._asdict() for i in messages],
        )
        content = completion.choices[0].message.content
        return content

質問に対して類似度の高い規約をVectorFrameから抽出します。

In [26]:
# データの読み込み
model_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/model-embeddings.csv")\
    .rename({"１ モデル就業規則":"text"}, axis=1)
# arrayとして認識させる
model_df['embedding'] = model_df['embedding'].apply(eval).apply(np.array)
model_df = VectorFrame(model_df)
# 質問
query = "年間休日は何日ですか"
# 関連規約の抽出
regulation = model_df.search_regulation(query)
regulation

,Unnamed: 0,text,embedding,token_length,similarities
271,271,（休日） 第２０条 休日は、次のとおりとする。① 日曜日 ② 年 月 日を起算日とする２週間...,"[0.00294488575309515, 0.0012329686433076859, 0...",135,0.875488
261,261,２ 休日は、原則として暦日（午前０時から午後１２時までの継続２４時間をいう。）で与えなければ...,"[-0.010599075816571712, -0.008891886100172997,...",168,0.869712
257,257,第２０条 休日は、次のとおりとする。 ① 土曜日及び日曜日 ② 国民の祝日（日曜日と重なった...,"[-0.0018275119364261627, 0.0001700313005130738...",111,0.867768
259,259,【第２０条 休日】 １ 労基法では何曜日を休日とするかあるいは国民の祝日を休日とするかについ...,"[0.010833602398633957, 0.0028621950186789036, ...",94,0.867758
268,268,（労働時間及び休憩時間） 第１９条 １週間の所定労働時間は、 年 月 日を起算日として、２週...,"[-0.001058968249708414, -0.005311335902661085,...",72,0.866729
243,243,４ 休日については、毎週少なくとも１回又は４週間を通じ４日以上与えなければなりません（労基法...,"[-0.01608872041106224, -0.001150691881775856, ...",56,0.865377
283,283,（休日） 第２０条 １年単位の変形労働時間制の適用を受ける労働者の休日については、１年単位の...,"[0.0009587518288753927, -0.0046020084992051125...",170,0.863532
284,284,２ １年単位の変形労働時間制を適用しない労働者の休日については、以下のとおり指定し、月間休日...,"[-0.000653393508400768, -0.003440316766500473,...",192,0.858441
75,75,第２４条（年次有給休暇の時間単位での付与）,"[-0.011348715983331203, -0.007791062351316214,...",24,0.858013
266,266,② 休日は労働者の労働義務のない日ですから、これを振り替える場合は、以下に示す措置が必要とな...,"[-0.005939829628914595, -0.01060493104159832, ...",164,0.854001


関連規約に基づいて、質問に対する回答をChatGPTが考える。

In [27]:
response = regulation.ask_regulation(query)
response

'規約には明確な数字が示されておらず、年間休日の日数は会社ごとに異なります。ただし、第20条によると、休日は次のとおりです。\n\n① 日曜日\n② 年月日を起算日とする2週間ごとの第2土曜日\n③ 国民の祝日（日曜日と重なったときは翌日）\n④ 年末年始（12月31日～1月3日）\n⑤ 夏季休日（8月13日～15日）\n⑥ その他会社が指定する日\n\nこのように、会社が指定する休日を含め、年間休日は会社によって異なります。関連規約番号は第20条です。'

## 対話形式でさらに質問に答えてもらう
前の質問と回答を与えて、"年間休日は何日ですか"に関連した規約についてさらに質問します。

In [28]:
messages = [
    Message(Role.User, query),
    Message(Role.Assistant, response)
]
next_query = "20条に規定される休日をリスト形式で列挙してください。"
next_response = regulation.ask_regulation(next_query, messages=messages)
next_response


'はい、以下が第20条に規定される休日のリストです。\n\n① 日曜日\n② 年月日を起算日とする2週間ごとの第2土曜日\n③ 国民の祝日（日曜日と重なったときは翌日）\n④ 年末年始（12月31日～1月3日）\n⑤ 夏季休日（8月13日～15日）\n⑥ その他会社が指定する日\n\n以上が第20条に規定される休日のリストです。'

In [29]:
messages = [
    Message(Role.User, query),
    Message(Role.Assistant, response),
    Message(Role.User, next_query),
    Message(Role.Assistant, next_response),
]
more_query = "20条には他に何が言及されてますか"
more_response = regulation.ask_regulation(more_query, messages=messages)
more_response

'第20条には、休日のリスト以外にも、休日の与え方についての規定が含まれています。具体的には、以下のような内容が規定されています。\n\n- 休日は原則として暦日で与えることが必要であるが、交替制勤務を導入する場合は継続した24時間で与えてもよい。\n- 休日は週に1回以上、または4週間を通じて4日以上与えなければならない。\n- 1年単位の変形労働時間制を適用する場合は、労使協定によって休日を指定することができる。\n\n以上が第20条に規定される、休日の与え方に関する内容です。'

In [30]:
messages = [
    Message(Role.User, query),
    Message(Role.Assistant, response),
    Message(Role.User, next_query),
    Message(Role.Assistant, next_response),
    Message(Role.User, more_query),
    Message(Role.Assistant, more_response),
]
another_query = "その与え方は日本の法律とどのような関連がありますか"
another_response = regulation.ask_regulation(another_query, messages=messages)
another_response


'第20条に規定される休日の与え方は、日本の労働基準法に基づいて定められています。具体的には、労働基準法第35条によって、休日は週に1回以上、または4週間を通じて4日以上与えなければならないと規定されています。また、労働基準法第39条によって、年次有給休暇を与えることが義務付けられており、年次有給休暇の日数は、労働者が1年間勤務した場合については、最低でも10日間とされています。\n\nしたがって、企業が第20条に規定される休日の与え方を実施することは、労働基準法に基づいた適法な行為であると言えます。ただし、労働基準法には最低限の規定が含まれているため、企業がより良い労働環境を提供するためには、これらの最低限の規定を上回るような取り組みが必要となります。'

いかがでしたでしょうか。本記事のまとめは以下です。

1. 就業規則のテキストデータを与えてベクトル化
2. 質問と各規約をベクトルとして内積を取り、類似順に抽出
3. ChatGPTのシステムプロンプトに質問に類似した規約を与えて、規約のエキスパートになってもらう。
4. 質問の回答を得る。
5. 質問と回答を次の質問に活かして更に回答を得る。

このようにしてChatGPTに就業規則の内容をわかりやすく答えてもらう事ができました。